In [2]:
%%capture
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [1]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import *
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Example for glyph + pinyin embedding

In [2]:
glyph = GlyphEmbedding('data/glyph_weight.npy')
pinyinV = PinyinEmbedding(30,'data/pinyin_map.json')
# Example
display(glyph(torch.tensor([0,1,2])))
display(pinyinV(torch.tensor([0,1])))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0421, -0.0327,  0.0096,  ...,  0.0711,  0.0417, -0.0224],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2629,  0.0584, -0.0224, -0.1282,  0.2653,  0.2522,  0.0376,  0.0921,
          0.0478,  0.1703,  0.2385, -0.1221,  0.1052,  0.2103, -0.0876, -0.0335,
         -0.2250,  0.2317, -0.2188,  0.2284, -0.0873, -0.0754, -0.0487, -0.1337,
          0.0356,  0.0856, -0.1935,  0.0685,  0.2788,  0.2450]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
# Example
_ = tokenizer('[UNK]辛业怠π[PAD]',return_tensors='pt')
display(_)
Anchibert(**_).last_hidden_state.shape

{'input_ids': tensor([[ 101,  100, 6789,  689, 2591,  223,    0,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

torch.Size([1, 8, 768])

In [ ]:
tokenizer.convert_ids_to_tokens(_['input_ids'][0,:])

['[CLS]', '[UNK]', '辛', '业', '怠', 'π', '[PAD]', '[SEP]']

### Load Necessary preproceeded Data

In [2]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [4]:
display(len(tr_in))
total_len = len(tr_in)
half = total_len//10
display(half)
train_split = int(0.95 * half)


770491

77049

In [5]:
trainSet = FusionDataset(tr_in[:train_split],tokenizer,\
                         glyph2ix,pinyin2ix,pos2ix,tr_out[:train_split],\
                         tr_pos_in[:train_split],tr_pos_out[:train_split],\
                         device=device) # use device if you want to load it gpu

73196it [00:32, 2282.21it/s]
73196it [00:33, 2200.30it/s]


In [6]:
valSet = FusionDataset(tr_in[train_split:half],tokenizer,\
                       glyph2ix,pinyin2ix,pos2ix,tr_out[train_split:half],\
                       tr_pos_in[train_split:half],tr_pos_out[train_split:half],\
                       device=device) # use device if you want to load it gpu

3853it [00:01, 2360.76it/s]
3853it [00:01, 2300.90it/s]


## Train Function

In [8]:
IF_WRITER = False
from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('pytorch_results')
import torch.optim as optim
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(1)

In [16]:
import time

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Check Time Cost for different Section

In [28]:
def train_epoch(model,dataLoader,optimizer,loss_function,
                bert, device,with_trans=True):
    """Train one epoch of the model"""
    epoch_loss = 0    
    model.train()
    
    embed_time = 0
    forward_time = 0
    cal_y_time = 0
    loss_opti_time = 0
    
    for Xsents_input_ids,Xsents_token_type_ids,\
        Xsents_attention_mask,Xsents_pinyin_ids,\
        Xsents_glyph_ids,Xsents_pos_ids,\
        Ysents_input_ids,Ysents_token_type_ids,\
        Ysents_attention_mask,Ysents_pinyin_ids,\
        Ysents_glyph_ids,Ysents_pos_ids,trueY in tqdm(dataLoader):
#         Ysents_glyph_ids,Ysents_pos_ids,trueY in dataLoader:

#         Xsents_input_ids = Xsents_input_ids.to(device)
#         Xsents_token_type_ids = Xsents_token_type_ids.to(device)
#         Xsents_attention_mask = Xsents_attention_mask.to(device)
#         Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
#         Xsents_glyph_ids = Xsents_glyph_ids.to(device)
#         Xsents_pos_ids = Xsents_pos_ids.to(device)
#         Ysents_input_ids = Ysents_input_ids.to(device)
#         Ysents_token_type_ids = Ysents_token_type_ids.to(device)
#         Ysents_attention_mask = Ysents_attention_mask.to(device)
#         Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
#         Ysents_glyph_ids = Ysents_glyph_ids.to(device)
#         Ysents_pos_ids = Ysents_pos_ids.to(device)
        
        s1 = time.time()
        Xword_embeddings = bert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                 )['last_hidden_state'].detach()


        Yword_embeddings = bert(Ysents_input_ids,      \
                                Ysents_token_type_ids, \
                                Ysents_attention_mask  \
                               )['last_hidden_state'].detach()
        s2 = time.time()
        embed_time += (s2-s1)
        inputs = {'Xword_embeddings':Xword_embeddings, \
                  'Xsents_pinyin_ids':Xsents_pinyin_ids, \
                  'Xsents_glyph_ids':Xsents_glyph_ids,\
                  'Xsents_pos_ids':Xsents_pos_ids,\
                  'Yword_embeddings':Yword_embeddings,\
                  'Ysents_pinyin_ids':Ysents_pinyin_ids, \
                  'Ysents_glyph_ids':Ysents_glyph_ids,\
                  'Ysents_pos_ids':Ysents_pos_ids, \
                  'device':device}
        
        if with_trans:
            inputs['Xpad_hidden_mask'] = (~ Xsents_attention_mask.bool()).detach()
            inputs['Ypad_hidden_mask'] = (~ Ysents_attention_mask.bool()).detach()
        
        # Clear gradients
        optimizer.zero_grad()
        
        outputs = model(**inputs) # [batch_size,max_length,output_dim]
        outputs = outputs.view(-1,outputs.shape[-1])   #[batch_size*max_length,output_dim]
        
        s3 = time.time()
        forward_time +=(s3-s2)
        
#         # get the true model output label from decoder input id
#         trueY = torch.zeros(Ysents_input_ids.shape,dtype=torch.long,device=device)
#         trueY[:,:-1] = Ysents_input_ids[:,1:]
#         # find the index of first padding return 0 if no_padding 
#         indices = torch.argmin(Ysents_attention_mask,dim=1).tolist()
        
#         for i, index in enumerate(indices):
#             trueY[i,index-1] = 102
        
        trueY = trueY.view(-1)
        
        s4 = time.time()
        cal_y_time += s4-s3
        
        # Calculate the loss
        loss = loss_function(outputs,trueY)
        
        # with torch.autograd.detect_anomaly():
        
        # Get gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        loss_opti_time += time.time()-s4
        epoch_loss += loss.item()
    return epoch_loss / len(dataLoader), embed_time,forward_time,cal_y_time,loss_opti_time

In [29]:
trainLoader = DataLoader(trainSet,batch_size=128,shuffle=True)
model.to(device)
Anchibert.to(device)
criterion = nn.NLLLoss(ignore_index=0)
lr = 0.1
optimizer = optim.Adam(model.parameters(),lr=lr)
res = train_epoch(model,trainLoader,optimizer, criterion,Anchibert,device)

100%|██████████| 1355/1355 [10:40<00:00,  2.11it/s]


In [30]:
res

(42.950739338653115,
 42.94941735267639,
 258.7296085357666,
 0.03971695899963379,
 316.05510091781616)

### Regular Training Section

In [9]:
def train_epoch(model,dataLoader,optimizer,loss_function,
                bert, device,with_trans=True):
    """Train one epoch of the model"""
    epoch_loss = 0    
    model.train()
    
    for Xsents_input_ids,Xsents_token_type_ids,\
        Xsents_attention_mask,Xsents_pinyin_ids,\
        Xsents_glyph_ids,Xsents_pos_ids,\
        Ysents_input_ids,Ysents_token_type_ids,\
        Ysents_attention_mask,Ysents_pinyin_ids,\
        Ysents_glyph_ids,Ysents_pos_ids,trueY in dataLoader:
#         Ysents_glyph_ids,Ysents_pos_ids in tqdm(dataLoader):

        Xsents_input_ids = Xsents_input_ids.to(device)
        Xsents_token_type_ids = Xsents_token_type_ids.to(device)
        Xsents_attention_mask = Xsents_attention_mask.to(device)
        Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
        Xsents_glyph_ids = Xsents_glyph_ids.to(device)
        Xsents_pos_ids = Xsents_pos_ids.to(device)
        Ysents_input_ids = Ysents_input_ids.to(device)
        Ysents_token_type_ids = Ysents_token_type_ids.to(device)
        Ysents_attention_mask = Ysents_attention_mask.to(device)
        Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
        Ysents_glyph_ids = Ysents_glyph_ids.to(device)
        Ysents_pos_ids = Ysents_pos_ids.to(device)
        
        Xword_embeddings = bert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                 )['last_hidden_state'].detach()


        Yword_embeddings = bert(Ysents_input_ids,      \
                                Ysents_token_type_ids, \
                                Ysents_attention_mask  \
                               )['last_hidden_state'].detach()
        
        inputs = {'Xword_embeddings':Xword_embeddings, \
                  'Xsents_pinyin_ids':Xsents_pinyin_ids, \
                  'Xsents_glyph_ids':Xsents_glyph_ids,\
                  'Xsents_pos_ids':Xsents_pos_ids,\
                  'Yword_embeddings':Yword_embeddings,\
                  'Ysents_pinyin_ids':Ysents_pinyin_ids, \
                  'Ysents_glyph_ids':Ysents_glyph_ids,\
                  'Ysents_pos_ids':Ysents_pos_ids, \
                  'device':device}
        
        if with_trans:
            inputs['Xpad_hidden_mask'] = (~ Xsents_attention_mask.bool()).detach()
            inputs['Ypad_hidden_mask'] = (~ Ysents_attention_mask.bool()).detach()
        
        # Clear gradients
        optimizer.zero_grad()
            
        outputs = model(**inputs) # [batch_size,max_length,output_dim]
        outputs = outputs.view(-1,outputs.shape[-1])   #[batch_size*max_length,output_dim]
                
        trueY = trueY.view(-1)
        
        # Calculate the loss
        loss = loss_function(outputs,trueY)
        
        # with torch.autograd.detect_anomaly():
        
        # Get gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(dataLoader)

def evaluate_epoch(model,dataLoader,loss_function,
                   bert, device,with_trans=True):
    """Evaluate one epoch of the model"""
    epoch_loss = 0    
    model.eval()
    with torch.no_grad():
         for Xsents_input_ids,Xsents_token_type_ids,\
            Xsents_attention_mask,Xsents_pinyin_ids,\
            Xsents_glyph_ids,Xsents_pos_ids,\
            Ysents_input_ids,Ysents_token_type_ids,\
            Ysents_attention_mask,Ysents_pinyin_ids,\
            Ysents_glyph_ids,Ysents_pos_ids,trueY in dataLoader:
#             Ysents_glyph_ids,Ysents_pos_ids in tqdm(dataLoader):

#             Xsents_input_ids = Xsents_input_ids.to(device)
#             Xsents_token_type_ids = Xsents_token_type_ids.to(device)
#             Xsents_attention_mask = Xsents_attention_mask.to(device)
#             Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
#             Xsents_glyph_ids = Xsents_glyph_ids.to(device)
#             Xsents_pos_ids = Xsents_pos_ids.to(device)
#             Ysents_input_ids = Ysents_input_ids.to(device)
#             Ysents_token_type_ids = Ysents_token_type_ids.to(device)
#             Ysents_attention_mask = Ysents_attention_mask.to(device)
#             Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
#             Ysents_glyph_ids = Ysents_glyph_ids.to(device)
#             Ysents_pos_ids = Ysents_pos_ids.to(device)

            Xword_embeddings = bert(Xsents_input_ids,      \
                                     Xsents_token_type_ids, \
                                     Xsents_attention_mask  \
                                     )['last_hidden_state'].detach()


            Yword_embeddings = bert(Ysents_input_ids,      \
                                    Ysents_token_type_ids, \
                                    Ysents_attention_mask  \
                                   )['last_hidden_state'].detach()

            inputs = {'Xword_embeddings':Xword_embeddings, \
                      'Xsents_pinyin_ids':Xsents_pinyin_ids, \
                      'Xsents_glyph_ids':Xsents_glyph_ids,\
                      'Xsents_pos_ids':Xsents_pos_ids,\
                      'Yword_embeddings':Yword_embeddings,\
                      'Ysents_pinyin_ids':Ysents_pinyin_ids, \
                      'Ysents_glyph_ids':Ysents_glyph_ids,\
                      'Ysents_pos_ids':Ysents_pos_ids, \
                      'device':device}

            if with_trans:
                inputs['Xpad_hidden_mask'] = (~ Xsents_attention_mask.bool()).detach()
                inputs['Ypad_hidden_mask'] = (~ Ysents_attention_mask.bool()).detach()


            outputs = model(**inputs) # [batch_size,max_length,output_dim]
            outputs = outputs.view(-1,outputs.shape[-1])   #[batch_size*max_length,output_dim]

            trueY = trueY.view(-1)

            # Calculate the loss
            loss = loss_function(outputs,trueY)
            epoch_loss += loss.item()
    return epoch_loss / len(dataLoader)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
def timeparser(elapse):
    mins = int(elapse/60)
    sec = int(elapse - (mins*60))
    return mins, sec

def train(model,trainSet,validSet,batch_size,lr, epoch,bert,name= None,
          with_trans= True,if_writer=False,optimizer_name= 'Adam',
          scheduleFactor=0.5,schedule_Patience=5,
          min_lr=1e-06, verbose=True, patience= 10):
    """
    Training a given neural network model
    Return a Best model object in state_dict() fashion
    
    @Para:
    model: pytorch training model
    trainSet: Tensor Dataset
    validSet: Tensor Dataset
    batch_size: batch_size
    lr: initial learning rate
    epoch: num of epoch
    bert: transformers.models.bert.modeling_bert.BertModel
    name: Model name
    with_trans: If True, a transformer padding mask will be generated as forward input. 
    
    if_writer: True for using tensorboard to trail
        the train loss and the valid loss at each epoch
    
    scheduleFactor: reduce factor for learning rate
    schedule_Patience:(int): Number of epochs with no improvement after
        which learning rate will be reduced. For example, if
        `patience = 2`, then we will ignore the first 2 epochs
        with no improvement, and will only decrease the LR after the
        3rd epoch if the loss still hasn't improved then.
        Default: 10.
    
    min_lr: min_lr
    verbose: True for print train loss and valid loss at each epoch
    patience:  Number of epochs with no improvement after
        which training will be stopped. For example, if
        `patience = 2`, then we will ignore the first 2 epochs
        with no improvement, and will only stops after the
        3rd epoch if the valid loss still hasn't improved then.
        Default: 5.
    """
     # Instantiate Train Loader and Valid Loader
    trainLoader = DataLoader(trainSet,batch_size=batch_size,shuffle=True)
    validLoader = DataLoader(validSet,batch_size=batch_size,shuffle=False)
    if if_writer:
        writer = SummaryWriter('pytorch_results')
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    bert.to(device)

    # Instantiate loss class
    criterion = nn.NLLLoss(ignore_index=0)
    
    # Iniantiate optimizer class
    lr = lr
    
    # Iniantiate optimizer class
    if optimizer_name:
        optimizer = optim.Adam(model.parameters(),lr=lr)
    else:
        optimizer = optim.SGD(model.parameters(),lr = lr, momentum=.9, nesterov=True)

     # Reduce on Loss Plateau Learning rate scheduler

    scheduler = ReduceLROnPlateau(optimizer=optimizer,factor=scheduleFactor,patience=schedule_Patience, min_lr=min_lr,verbose=verbose)
    
    # Instantiate Best model and Best Valid Loss
    best_valid_loss = float('inf')
    
    valid_patience_counter = 0
    
    n_epoch = epoch
    train_losses = []
    valid_losses = []
    for epoch in tqdm(range(n_epoch)):
        
        start_time = time.time()
        
        # train model (average)
        train_loss = train_epoch(model,trainLoader,optimizer,
                                 criterion,bert,device,with_trans)
        
        # Decay Learning Rate
        scheduler.step(train_loss)
        
        ##############
        # validation #
        ##############
        valid_loss = evaluate_epoch(model,validLoader,criterion,
                                               bert,device,with_trans)
        mins,secs = timeparser(time.time()-start_time)
        
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        if if_writer:
            writer.add_scalar(f"{name} Train Loss", train_loss, epoch)
            writer.add_scalar(f"{name} Valid Loss", valid_loss,epoch)
            writer.add_scalars(f"{name} Loss/Epoch", {'train': train_loss,
                                                    'valid': valid_loss},
                                epoch+1)
        
        if verbose:
            # Print Learning Rate and Loss for ecach Loss
            print(f'Epoch: {epoch+1:02} | Epoch Time: {mins}m {secs}s')
            print(f'\tTraining Loss: {train_loss:.5f} \tValidation Loss: {valid_loss:.5f}')
        
        # Compare the Valid Loss with Best Valid Loss
        if valid_loss< best_valid_loss:
            # reset counter
            valid_patience_counter = 0
            best_model = deepcopy(model.state_dict())
            best_valid_loss = valid_loss
        else:
            valid_patience_counter += 1
            if valid_patience_counter == patience:
                if if_writer:
                    writer.flush()
                    writer.close()
                return best_model,train_losses,valid_losses
    if if_writer:
        writer.flush()
        writer.close()
    return best_model, train_losses,valid_losses


In [12]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer, Anchi_Decoder,Anchi_Transformer

In [13]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6,
    'output_dim':21128 # fixed
}
config2 = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_encoder_layers':6,
    'num_decoder_layers':6,
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1
}

In [17]:
model = Fusion_Anchi_Trans_Decoder(config)

In [ ]:
# model2 = Fusion_Anchi_Transformer(config2)

In [ ]:
# model3 = FusionEmbedding(config)
# model3

FusionEmbedding(
  (position_embeddings): Embedding(50, 768)
  (glyph_embeddings): GlyphEmbedding(
    (embedding): Embedding(9110, 576)
  )
  (pinyin_embeddings): PinyinEmbedding(
    (embedding): Embedding(1297, 30, padding_idx=0)
  )
  (pos_tag_embeddings): Embedding(30, 10, padding_idx=0)
  (fc): Linear(in_features=1384, out_features=768, bias=True)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
name = 'fusion_decoder_64_1_60_'
best_model,train_losses,valid_losses = train(model,trainSet,valSet,
                                          batch_size=64,lr=0.1,
                                          epoch=60,bert=Anchibert,
                                          name= name,
                                          with_trans=True,if_writer=False,
                                          optimizer_name='Adam',scheduleFactor=0.5,
                                          schedule_Patience=5,min_lr=1e-06,
                                          verbose=True,patience=10
                                         )

  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
torch.save(best_model,f'{name}.pt')
with open(f'{name}_losses.pt','w') as f:
    pickle.dump((train_losses,valid_losses),f)

In [ ]:
model.load_state_dict(best_model)

### Way to generate new decoder input from raw sentence 

记得把model output 中 不存在 glyph dict的词以及'[SEP]' 和'[PAD]'给转换成 '_'。

In [ ]:
sents = tr_out[:2]
sents

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['晨', '露', '润', '花', '花', '更', '红'], ['万', '方', '乐', '奏', '有', '于', '阗']]

In [ ]:
FusionDataset.prepare_sequence(sents=sents, tokenizer=tokenizer,
                               glyph2ix=glyph2ix,pinyin2ix=pinyin2ix,
                               pos2ix=pos2ix,encode=False,skip_error=False,
                               device=device) # use device if you want to load it gpu

Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
2it [00:00, 83.52it/s]


([tensor([ 101, 3247, 7463, 3883, 5709, 5709, 3291, 5273], dtype=torch.int32),
  tensor([ 101,  674, 3175,  727, 1941, 3300,  754,  100], dtype=torch.int32)],
 [tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
  tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)],
 [tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32),
  tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)],
 [tensor([  0, 717,  59, 873, 579, 579, 217, 334], dtype=torch.int32),
  tensor([   0,  187,  476,  501,  807, 1178,  263, 1002], dtype=torch.int32)],
 [tensor([   0, 5656, 1568, 3172, 3915, 3915, 3367, 8111], dtype=torch.int32),
  tensor([   0, 1729, 3171, 4526, 2058, 8413, 3025, 6866], dtype=torch.int32)],
 [tensor([ 0,  4,  4,  4,  4,  4, 16, 12], dtype=torch.int32),
  tensor([ 0, 26, 26, 26, 26,  9, 27, 27], dtype=torch.int32)])

#### Section for testing train and evaluate function

In [7]:
tempSet = FusionDataset(tr_in[:1000],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[:1000],tr_pos_in[:1000],tr_pos_out[:1000])
tempLoader = DataLoader(tempSet,batch_size=64,shuffle=False)

1000it [00:00, 2223.45it/s]
1000it [00:00, 2323.16it/s]


In [9]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr=0.1)
criterion = nn.NLLLoss(ignore_index=0)

In [ ]:
%%capture
Anchibert.to(device)

In [12]:
model = Anchi_Transformer(config2).to(device)

In [20]:
model = Anchi_Decoder(config).to(device)

In [25]:
model = Fusion_Anchi_Transformer(config2).to(device)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
model = Fusion_Anchi_Trans_Decoder(config).to(device)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
res = train_epoch(model,tempLoader,optimizer,criterion,Anchibert,device,with_trans=True)

100%|██████████| 16/16 [00:03<00:00,  4.62it/s]


In [41]:
res = evaluate_epoch(model,tempLoader,criterion,Anchibert,device,with_trans=True)

100%|██████████| 16/16 [00:02<00:00,  7.56it/s]
